In [9]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy.spatial.distance import cdist

In [10]:
# Set dimensions
K1 = 1  # Number of observations for Group 1
K2 = 1  # Number of observations for Group 2
K = K1 + K2  # Total number of observations
D = 6  # Total latent dimensions
T = 50  # Number of time points
t = np.linspace(0, 10, T)  # Time intervals

# Partition D into shared and independent parts
d_s = 2  # Shared latent dimension
d_1 = 2  # Independent latent dimension for group 1
d_2 = 2  # Independent latent dimension for group 2

# Pick hyperparameters
rho = 1.0  # Scale for GP kernel
l = 1.0  # Length scale for GP kernel
nu = 0.1  # Noise variance for observations

In [28]:
# Define factor loadings
A_1 = np.random.randn(K1, d_1)  # Group 1 independent loadings
A_2 = np.random.randn(K2, d_2)  # Group 2 independent loadings
A_s1 = np.random.randn(K1, d_s)  # Group 1 shared loadings
A_s2 = np.random.randn(K2, d_s)  # Group 2 shared loadings

# Combine into block matrix
A = np.block([[A_s1, A_1, np.zeros((K1, d_2))],  # Group 1
              [A_s2, np.zeros((K2, d_1)), A_2]])  # Group 2

print(A.shape)  # Should be (K, D), i.e., (2, 6)
A

(2, 6)


array([[ 0.61305669, -1.58412279,  0.85990261,  2.76702061,  0.        ,
         0.        ],
       [ 0.66647931, -1.2414752 ,  0.        ,  0.        , -0.43058378,
        -0.23816071]])

In [29]:
# Generate latent variables (z)
def kernel_function(t1, t2, rho, l):
    """Squared exponential kernel."""
    dist_sq = cdist(t1.reshape(-1, 1), t2.reshape(-1, 1), metric='sqeuclidean')
    return rho * np.exp(-dist_sq / (2 * l ** 2))

# Create kernel matrix for GP
K_t = kernel_function(t, t, rho, l)

# Draw latent variables for each group and shared part
z_shared = np.random.multivariate_normal(np.zeros(T), K_t, size=d_s).T  # Shared
z_1 = np.random.multivariate_normal(np.zeros(T), K_t, size=d_1).T  # Group 1
z_2 = np.random.multivariate_normal(np.zeros(T), K_t, size=d_2).T  # Group 2

# Combine all latent variables into one matrix
Z = np.hstack([z_shared, z_1, z_2])  # Shape (T, D)
print(Z.shape)

(50, 6)


In [30]:
# Generate observations (x)
E = np.random.multivariate_normal(np.zeros(K), nu * np.eye(K), size=T)  # Noise
X = Z @ A.T + E  # Shape (T, K)

In [31]:
# Generate spike observations (y)
# Poisson lambda is exp(X)
Lambda = np.exp(X)
Y = np.random.poisson(Lambda)  # Spike observations


In [32]:
print("Latent variables (Z):", Z.shape)
print("Observations (X):", X.shape)
print("Spike observations (Y):", Y.shape)

Latent variables (Z): (50, 6)
Observations (X): (50, 2)
Spike observations (Y): (50, 2)
